In [15]:
import pandas as pd
import re
import os
comment_types = ["no_comments", "comments", "added_comments"]

In [16]:
# import numpy as np
# import random
# random.seed(42)

# path = f"./fold0/no_comments"
# project = 'commons-jexl3-3.2.1-src'
# project_data = pd.read_csv(f"{path}/{project}/test_stats.csv")
# pass_rate = 0.7634
# df = project_data.copy()
# num_rows = len(df)
# coin_accuracy = []
# coin_f1 = []

# iterations = int(np.ceil(10000 / num_rows))

# for i in range(iterations):
#     coin_simulation = [1 if (random.random() < pass_rate) else 0 for i in range(num_rows)]
#     df["coin_simulation"] = coin_simulation
#     coin_tp = 0
#     coin_fn = 0
#     coin_tn = 0
#     coin_fp = 0

#     for i in range(num_rows):

#         # row[0] is the prediction, Row[1] is the actual
#         # predicted and actual pass
#         if df.loc[i, "Actual Label"] == 1 and df.loc[i, "coin_simulation"] == 1:
#             coin_tp += 1
#         # predicted fail and actual pass
#         elif df.loc[i, "Actual Label"] == 1 and df.loc[i, "coin_simulation"] == 0:
#             coin_fn += 1
#         # predicted and actual fail
#         elif df.loc[i, "Actual Label"] == 0 and df.loc[i, "coin_simulation"] == 0:
#             coin_tn += 1
#         # predicted pass and actual fail
#         elif df.loc[i, "Actual Label"] == 0 and df.loc[i, "coin_simulation"] == 1:
#             coin_fp += 1
#     accuracy = (coin_tp + coin_tn) / num_rows if num_rows > 0 else 0
#     f1 = (2 * coin_tp) / (2 * coin_tp + coin_fp + coin_fn) if (2 * coin_tp + coin_fp + coin_fn) > 0 else 0
#     coin_accuracy.append(accuracy)
#     coin_f1.append(f1)

# average_coin_accuracy = np.round(np.mean(coin_accuracy), 4)
# average_coin_f1 = np.round(np.mean(coin_f1), 4)


# Overall Stats

In [17]:
for comment_type in comment_types:
    for val in ["all", "50", "25", "10", "05"]:
        df = pd.read_csv(f"./fold0/{comment_type}/project_stats_{val}.csv")
        # Simplify project names
        df["project"] = df.apply(lambda row: re.split(r"-\d", row["project"])[0], axis=1)
        df.rename(
            columns={
                "accuracy": "acc.",
                "pass_accuracy": "pass_acc.",
                "fail_accuracy": "fail_acc.",
                "pass_rate": "dataset_pass_%",
                "fail_rate": "dataset_fail_%",
                "accuracy_improvement": "acc_\Delta",
                "coin_accuracy": "coin_acc.",
                "out_vocab_C_ratio": "missing_C_%",
                "out_vocab_T_ratio": "missing_T_%",
                "out_vocab_combined_ratio": "missing_token_%",
            },
            inplace=True,
        )

        table1 = df[
            [
                "project",
                "N",
                "dataset_pass_%",
                "dataset_fail_%",
                "missing_C_%",
                "missing_T_%",
                "missing_token_%",
            ]
        ]

        table2 = df[
            [
                "project",
                "acc_\Delta",
                "acc.",
                "pass_acc.",
                "fail_acc.",
                "f1",
                "coin_acc.",
                "coin_f1",
                "tp",
                "fn",
                "tn",
                "fp",
            ]
        ]

        if val == "all":
            table1.to_latex(
                f"./latex/{comment_type}/toga_dataset_stats_{val}.tex",
                index=False,
                caption=f"TOGA* Dataset Statistics",
                label=f"tab:toga_stats_{val}",
            )
            table2.to_latex(
                f"./latex/{comment_type}/toga_results_{val}.tex",
                index=False,
                caption=f"SEER Results on TOGA*",
                label=f"tab:toga_results_{val}",
            )
        else:
            table2 = df[
                [
                    "project",
                    "N",
                    "acc_\Delta",
                    "acc.",
                    "pass_acc.",
                    "fail_acc.",
                    "f1",
                    "coin_acc.",
                    "coin_f1",
                    "tp",
                    "fn",
                    "tn",
                    "fp",
                ]
            ]

            table1.to_latex(
                f"./latex/{comment_type}/toga_dataset_stats_{val}.tex",
                index=False,
                caption=f"TOGA* Dataset Statistics, restricted to minimum {val}\% of tokens present",
                label=f"tab:toga_stats_{val}",
            )
            table2.to_latex(
                f"./latex/{comment_type}/toga_results_{val}.tex",
                index=False,
                caption=f"SEER Results on TOGA*, restricted to minimum {str(100-int(val))}\% of tokens present",
                label=f"tab:toga_results_{val}",
            )

In [18]:
# Fixing some LaTeX issues
for comment_type in comment_types:

    for filename in os.listdir(f"./latex/{comment_type}"):
        with open(f"./latex/{comment_type}/{filename}", "r+") as f:
            text = f.read()
            text = re.sub(r"\\textbackslash Delta", "$\Delta$", text)
            text = re.sub("table", "table*", text)
            f.seek(0)
            f.write(text)
            f.truncate()

# similarity

In [ ]:
df_common_unique = pd.read_csv(f"./similarity_analysis/similarity_unique_mut.csv")
# Simplify project names
df_common_unique["triplets"] = df_common_unique.apply(lambda row: re.split(r"-\d", row["triplets"])[0], axis=1)

df_common_unique.rename(
    columns={
        "phase2": "SEER",
        "triplets": "TOGA*",
        "triplets_unique_count": "TOGA*_count",
        "phase2_unique_count": "SEER_count",
    },
    inplace=True,
)

df_common_unique[["SEER", "TOGA*", "SEER_count", "TOGA*_count"]].to_latex(
    f"./latex/common_projects_unique.tex",
    index=False,
    caption=f"Unique Methods Under Test",
    label=f"tab:common_unique_MUT",
)